This is an exploration of building a time-based graph of activities based
on conditional commit nomenclature in the commit messages of a project.


In [15]:
import re
from datetime import datetime, timedelta

import git
import pandas as pd
import plotly.express as px

In [16]:

pattern = re.compile(r'^(\w+)[\!(:]')

In [17]:
wfx_repo = "/Users/timottinger/Projects/siemens-wfx"
goreleaser = "/Users/timottinger/Projects/goreleaser"
repo = git.Repo(goreleaser)

In [18]:
def reason_by_date(repo: git.Repo, on_or_after: datetime):
    for commit in repo.iter_commits():
        if commit.authored_datetime < on_or_after:
            continue
        parse = pattern.match(commit.message)
        reason = parse.group(1) if parse else commit.message.split()[0]
        date = commit.authored_datetime.date()
        yield (date, reason)


Don't try to graph the whole history, just the past year or so

In [19]:
earliest = datetime.today().astimezone() - timedelta(weeks=52)
data_set = reason_by_date(repo, earliest)
df = pd.DataFrame(data_set, columns=['date', 'reason'])

In [20]:
figure = px.bar(df.tail(30), x='date', color='reason', labels="date")
figure.show()